In [48]:
import heapq as hq

class Graph:

    def __init__(self, vertices):
        self.V = vertices
        self.graph = {}
        
        # store the minimum cycles of each node
        self.cycles = {}
        # store the dijkstra between any two node
        self.dijkstras = {}
                           
    # Dijkstra algorithm by heap data structure
    def dijkstra(self, start):
        distances = {}
        heap = [(0, start)]

        while heap:
            dist, node = hq.heappop(heap)
            if node in distances:
                continue  # Already encountered before
            # We know that this is the first time we encounter node.
            #   As we pull nodes in order of increasing distance, this 
            #   must be the node's shortest distance from the start node.
            distances[node] = dist
            for neighbor, weight in self.graph[node]:
                if neighbor not in distances:
                    hq.heappush(heap, (dist + weight, neighbor))

        return distances
    
    # Dijkstra algorithm for finding the minimum path between two node. 
    # During running algorithm, if the minimum path between two nodes defined, the algorithm will be terminated. 
    def dijkstraST(self, g, start, trg):
        distances = {}
        heap = [(0, start)]

        while heap:
            dist, node = hq.heappop(heap)
            
            # if we find the target node, terminate algorithm
            if node == trg:
                return dist
            
            if node in distances:
                continue  # Already encountered before
            # We know that this is the first time we encounter node.
            #   As we pull nodes in order of increasing distance, this 
            #   must be the node's shortest distance from the start node.
            distances[node] = dist
            for neighbor, weight in g[node]:
                if neighbor not in distances:
                    hq.heappush(heap, (dist + weight, neighbor))
                    
        # we could not find path between two nodes, so they are not connected. 
        return float('inf')

    # finding the path between any two node by Dijkstra algorithm
    # then store the result in self.dijkstras
    def AllDijkstra(self):
        for c in self.graph:
            self.dijkstras[c] = g.dijkstra(c)
        return self.dijkstras
    
    
    def leastCycleDijkstra(self, v):
        least_cycle = float('inf')
        for i in range(len(self.graph[v])):
            
            
            label = self.graph[v][i][0]
            weight = self.graph[v][i][1]
            index_delete1 = -1
            index_delete2 = -1
            
            temp_graph = {}
            for i in self.graph:
                temp_graph[i] = self.graph[i].copy()
                
            for i in range(len(temp_graph[v])):
                if temp_graph[v][i][0] == label:
                    index_delete1 = i
                    break
            del temp_graph[v][index_delete1]
            
            for i in range(len(temp_graph[label])):
                if temp_graph[label][i][0] == v:
                    index_delete2 = i
                    break
            del temp_graph[label][index_delete2]        
            
            temp = self.dijkstraST(temp_graph, v, label)
            
            cycle = temp + weight
            if cycle < least_cycle:
                least_cycle = cycle
            del temp_graph
            
        return least_cycle
    
    def find_lca(self, x, parents_x, y, parents_y):
        if parents_x == [] or x in parents_y:
            return x
        elif parents_y == [] or y in parents_x:
            return y
        elif len(parents_x) < len(parents_y):
            p_y = parents_y[:len(parents_x)]
            p_x = parents_x
        else:
            p_y = parents_y
            p_x = parents_x[:len(parents_y)]
        for i in range(len(p_y) -1, -1, -1):
            
            if p_y[i] == p_x[i]:
                return p_x[i]
        
        
    # the optimum method for finding girth of a graph
    def optimumCompositeDistance(self, start):
        composite_distance = float('inf')
        distances = {}
        heap = [(0, start, [])]

        while heap:
            dist, node, parents = hq.heappop(heap)
            print(dist, node, parents)
            if node in distances or composite_distance <= dist:
                continue  # Already encountered before or we cannot find the minimum cycle after this node
                
            # We know that this is the first time we encounter node.
            #   As we pull nodes in order of increasing distance, this 
            #   must be the node's shortest distance from the start node.
            distances[node] = [dist, parents]
            
            for neighbor, weight in self.graph[node]:
                # a cycle detected
                
                if neighbor in distances and neighbor != parents[-1]:
                    print(neighbor, parents[-1])
                    lca = self.find_lca(neighbor, distances[neighbor][1], node, parents)
                    print(lca)
                    temp_cycle = dist+ weight+distances[neighbor][0] - distances[lca][0]
                    if temp_cycle < composite_distance:
                        composite_distance = temp_cycle
                elif neighbor not in distances:
                    # print(neighbor, distances)
                    temp = parents + [node]
                    hq.heappush(heap, (dist + weight, neighbor, temp))

        return composite_distance
    
    
    def optimumGirth(self):
        least_cycle = float('inf')
        composite_distances = {}
        
        for node in self.graph.keys():
            if node not in composite_distances: 
                # composite distance = [weight_composite_distance, first_encountered_node_in_cycle, dist_till_first_encountered_node_in_cycle]
                composite_distance = [float('inf'), -1, -1]
                distances = {}
                # creating a min-heap for dijkstra search
                heap = [(0, node, -1)]
                while heap:
                    # pop from heap, which has the least dist
                    dist, nodeTarget, parent = hq.heappop(heap)

                    if nodeTarget in distances or composite_distance[0] <= dist:
                        continue  # Already encountered before or we cannot find the minimum cycle after this node

                    # We know that this is the first time we encounter node.
                    #   As we pull nodes in order of increasing distance, this 
                    #   must be the node's shortest distance from the start node.
                    distances[nodeTarget] = dist
                    for neighbor, weight in self.graph[nodeTarget]:
                        # we found the composite_distance of this neighbor before
                        # so we just use the information to speed up our search
                        if neighbor in composite_distances:
                            temp_comp = composite_distances[neighbor][0] + weight + dist
                            if temp_comp < composite_distance[0]:
                                composite_distance[0] = temp_comp
                                
                        # a cycle detected
                        elif neighbor in distances and neighbor != parent:
                            temp_cycle = dist+ weight+distances[neighbor]
                            if temp_cycle < composite_distance[0]:
                                composite_distance[0] = temp_cycle
                                composite_distance[1] = neighbor
                                composite_distance[2] = distances[neighbor]
                        # We have not faced this node before, so we need to process it. 
                        # Accordingly, we push it into our heap
                        elif neighbor not in distances:
                            hq.heappush(heap, (dist + weight, neighbor, nodeTarget))
                            
                composite_distances[node] = composite_distance
                
                if composite_distance[0] < least_cycle:
                    least_cycle = composite_distance[0]
        return least_cycle
        
        
    def AllCyclesDijkstra(self):
        
        for c in self.graph:
            self.cycles[c] = g.leastCycleDijkstra(c)
        return self.cycles
    
    def compositeDistance(self, x, c):
        return self.dijkstras[x][c] + self.cycles[c]
        
    def compositeDistanceX(self, x):
        least_composite = float('inf')
        for c in range(self.V):
            if self.compositeDistance(x, c) < least_composite:
                least_composite = self.compositeDistance(x, c)
        return least_composite
    
    def girthComposite(self):
        least_composite = float('inf')
        for x in range(self.V):
            cdx = self.compositeDistanceX(x)
            if  cdx < least_composite:
                least_composite = cdx
        return least_composite
    
    def girth(self):
        return min(self.cycles, key=self.cycles.get)
            

g = Graph(8)

g.graph = {
    "a": [("b", 8), ("c", 3), ("d", 6)],
    "b": [("a", 8), ("e", 5)],
    "c": [("a", 3), ("d", 2)],
    "d": [("a", 6), ("c", 2), ("e", 15), ("g", 3)],
    "e": [("b", 5), ("d", 15), ("f", 5)],
    "f": [("e", 5), ("g", 3), ("h", 6)],
    "g": [("d", 3), ("f", 3), ("h", 4)],
    "h": [("g", 4), ("f", 6)]
}



# print(g.AllDijkstra())

# print(g.dijkstra(0))
# d(x,y)= g.dijkstra(x)[y]

# print(g.AllCyclesDijkstra())

# l(c)= g.leastCycleForEachNode(c)
# g.leastCycleForEachNode(0)
# g.leastCycleDijkstra(2)

  
# d+(x,c) = d(x,c) + l(c)= g.compositeDistance(x, c)
# g.compositeDistance(0, 3)

# d+(x)= min d+(x, c) = g.compositeDistanceX(x)
# g.compositeDistanceX(0)

# print(g.girth())

print(g.optimumCompositeDistance('b'))
# print(g.optimumGirth())


0 b []
5 e ['b']
8 a ['b']
10 f ['b', 'e']
11 c ['b', 'a']
13 d ['b', 'a', 'c']
a c
a
e c
b
13 g ['b', 'e', 'f']
d f
b
14 d ['b', 'a']
16 g ['b', 'a', 'c', 'd']
16 h ['b', 'e', 'f']
g f
f
17 h ['b', 'e', 'f', 'g']
20 d ['b', 'e']
19


In [ ]:
n = {'a': }